
## HySWAN Metamodel: Waves dataset propagation to a point.

This is a very simplified example of HySWAN toolbox. 

A waves series dataset will be propagated to a coastal point using the statistical-numerical metamodel, reducing number of SWAN cases to execute, and therefore computational resources required.

SWAN scenario is a demo case based on Roi-namur (Kwajalein atoll).

<strong>Workflow</strong>:


- Generate a waves subset using Maximum Dissimilarity Algorithm (MDA) classification from waves series dataset.
- Numerical solve waves propagations, using MDA waves subset as SWAN boundary conditions. Store propagated waves at a point.
- Feed SWAN input / output to a Radial Basis Functions model.
- Use RBF interpolator to statistically reconstruct propagation for the entire waves series dataset.


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# dev library
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# swan wrap module
from hywaves.swan.wrap import SwanProject, SwanMesh, SwanWrap_STAT
from hywaves.swan.plots.stationary import scatter_maps

# statistics
from hywaves.statistical.mda import MaxDiss_Simplified_NoThreshold
from hywaves.statistical.rbf import RBF_Reconstruction, RBF_Validation

# plotting
from hywaves.statistical.plots.mda import Plot_MDA_Data



## Database

In [ ]:
# --------------------------------------
# data
p_data = op.abspath(op.join(os.path.abspath(''), '..', '..', 'data'))
p_demo = op.join(p_data, 'demo', 'nb_demo_roi')

# swan main and nest1 mesh
p_swan_ext = op.join(p_demo, 'Roinamur_ext_bottom.dat')
p_swan_int = op.join(p_demo, 'Roinamur_int_bottom.dat')

# waves data
p_waves = op.join(p_demo, 'waves_historical.nc')

# roi shore (for plots)
p_shore = op.join(p_demo, 'shore.npy')
np_shore = np.load(p_shore)


In [ ]:
# load waves dataset
dataset = xr.open_dataset(p_waves)  # > 300000 waves cases

# prepare waves data for SWAN STATIONARY 
dataset = dataset[['Hs', 'Tp', 'Dir']].to_dataframe()[1:]
dataset.rename(columns={"Hs":"hs", "Tp":"tp", "Dir": "dir"}, inplace=True)

print(dataset)



## Maximum Dissimilarity Algorithm (MDA) classification

In [ ]:
# --------------------------------------
# Set MDA parameters

# variables to use
vns = ['hs', 'tp', 'dir']

# subset size, scalar and directional indexes
n_subset = 125            # subset size (very small for this conceptual example)
ix_scalar = [0, 1]        # hs, tp
ix_directional = [2]      # dir


In [ ]:
# MDA algorithm
sel = MaxDiss_Simplified_NoThreshold(dataset[vns].values[:], n_subset, ix_scalar, ix_directional)
subset = pd.DataFrame(data=sel, columns=vns)

# plot classification
Plot_MDA_Data(dataset, subset);


In [ ]:
# fill/fix subset variables for SWAN execution

subset['spr'] = 40  # set directional spread
subset.rename(columns={'tp':'per'}, inplace=True)  # rename Tp for swan wrapper


## SWAN: Project

In [ ]:
# --------------------------------------
# SWAN project 

p_proj = op.join(p_data, 'projects')  # swan projects main directory
n_proj = 'nb_01_mdarbf'               # project name

sp = SwanProject(p_proj, n_proj)


## SWAN: main and nested grid descriptions

In [ ]:
# --------------------------------------
# SWAN main mesh
main_mesh = SwanMesh()

# depth grid description (input bathymetry grid)
main_mesh.dg = {
    'xpc': 166.6159,  # x origin
    'ypc': 8.4105,    # y origin
    'alpc': 0,        # x-axis direction 
    'xlenc': 1.3722,  # grid length in x
    'ylenc': 1.175,   # grid length in y
    'mxc': 137,       # number mesh x
    'myc': 118,       # number mesh y
    'dxinp': 0.01,    # size mesh x
    'dyinp': 0.01,    # size mesh y
}

# depth value
main_mesh.depth = np.loadtxt(p_swan_ext)

# computational grid description
main_mesh.cg = main_mesh.dg.copy()

sp.set_main_mesh(main_mesh)


# --------------------------------------
# SWAN nest1 mesh
mesh_nest1 = SwanMesh()

# depth grid description
mesh_nest1.dg = {
    'xpc': 167.4059,
    'ypc': 9.3505,
    'alpc': 0,
    'xlenc': 0.12,
    'ylenc': 0.08,
    'mxc': 120,
    'myc': 80,
    'dxinp': 0.001,
    'dyinp': 0.001,
}

# depth value
mesh_nest1.depth = np.loadtxt(p_swan_int)

# computational grid description
mesh_nest1.cg = mesh_nest1.dg.copy()

sp.set_nested_mesh_list([mesh_nest1])


## SWAN: input parameters

In [ ]:
# --------------------------------------
# SWAN parameters (sea level, jonswap gamma...)

input_params = {
    'set_level': 0,
    'set_convention': 'NAUTICAL',

    'coords_mode': 'SPHERICAL',
    'coords_projection': 'CCM',
    
    'boundw_jonswap': 10,
    'boundw_period': 'PEAK',

    'boundn_mode': 'CLOSED',

    'physics':[
        'FRICTION JONSWAP',
        'BREAKING',
    ],

    'numerics':[
        'OFF QUAD',
    ]
}
sp.set_params(input_params)


## SWAN: Build and launch waves cases

In [ ]:
# SWAN project wrapper
sw = SwanWrap_STAT(sp)

# build and launch cases
sw.build_cases(subset)
sw.run_cases()


## SWAN: output visualization

In [ ]:
# extract output for main mesh
xds_out = sw.extract_output()
print(xds_out)

# plot scatter maps
scatter_maps(xds_out, n_cases=25, np_shore=np_shore);


In [ ]:
# extract output for nested mesh
xds_out_n1 = sw.extract_output(mesh=sp.mesh_nest1)
print(xds_out_n1)

# plot scatter maps
scatter_maps(xds_out_n1, n_cases=25, np_shore=np_shore);


## SWAN: extract output at a selected point 

In [ ]:
# point to reconstruct 
ix_lon, ix_lat = 62, 66

# Extract propagated waves at point (from nest1 mesh)
target = xds_out_n1.isel({'lon':ix_lon, 'lat':ix_lat}).to_dataframe()

# this will be used as RBF target while fitting statistical model
print(target)


##  Radial Basis Function reconstruction

In [ ]:
# Prepare data

subset.rename(columns={'per':'tp'}, inplace=True)  # reset subset variable name

# select dataset and subset variables
vns_ds = ['hs', 'tp', 'dir']

# select target variables
vns_tgt = ['Hsig', 'TPsmoo', 'Dir']

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]      # scalar (hs, tp)
ix_directional_subset = [2]   # directional (dir)

# target - scalar / directional indexes
ix_scalar_target = [0,1]      # scalar (Hsig, Tpsmoo, Dir)
ix_directional_target = [2]   # directional (Dir)


In [ ]:
# Obtain statistical propagation output for the entire waves dataset
output = RBF_Reconstruction(
    subset[vns_ds].values, ix_scalar_subset, ix_directional_subset,
    target[vns_tgt].values, ix_scalar_target, ix_directional_target,
    dataset[vns_ds].values,
)

# mount output as pandas.DataFrame
output = pd.DataFrame(data=output, columns=vns_tgt, index=dataset.index)

print(output)
